In [5]:
use Modern::Perl;
use HackaMol;
use POSIX;
use YAML::XS qw(Dump DumpFile);
use Time::HiRes qw(time);



In [6]:
my $mol = HackaMol->new->read_file_mol('/users/chem_student/cystine_xtals.xyz');


HackaMol::Molecule=HASH(0x7fed5282a230)


In [7]:
say $mol->t; 
say $mol->tmax;
$mol->print_xyz;
$mol->t(0);
$mol->print_xyz;

0
7472
12

  N  -2.345483  -0.247640   3.482699
  C  -1.467483  -0.186640   2.314699
  C  -0.286483   0.773360   2.434699
  O   0.865517   0.397360   2.204699
  C  -2.293483   0.211360   1.088699
  S  -1.313483   0.705360  -0.368301
  N   3.407517  -1.065640  -1.425301
  C   2.041517  -0.605640  -1.656301
  C   1.986517   0.918360  -1.705301
  O   1.296517   1.504360  -2.541301
  C   1.109517  -1.126640  -0.565301
  S  -0.637483  -1.074640  -1.078301
12

  N  -2.345483  -0.247640   3.482699
  C  -1.467483  -0.186640   2.314699
  C  -0.286483   0.773360   2.434699
  O   0.865517   0.397360   2.204699
  C  -2.293483   0.211360   1.088699
  S  -1.313483   0.705360  -0.368301
  N   3.407517  -1.065640  -1.425301
  C   2.041517  -0.605640  -1.656301
  C   1.986517   0.918360  -1.705301
  O   1.296517   1.504360  -2.541301
  C   1.109517  -1.126640  -0.565301
  S  -0.637483  -1.074640  -1.078301


GLOB(0x7fed4b02a9d8)


In [28]:
my $dihe3 = HackaMol::Dihedral->new(
    atoms=> [
         $mol->get_atoms(4), 
         $mol->get_atoms(5), 
         $mol->get_atoms(11),
         $mol->get_atoms(10) 
    ]
    ); 

my $dihe1 = HackaMol::Dihedral->new(
    atoms=> [ 
         $mol->get_atoms(1), 
         $mol->get_atoms(4), 
         $mol->get_atoms(5), 
         $mol->get_atoms(11),
        ]
        ); 
my $dihe1p = HackaMol::Dihedral->new(
    atoms=> [ 
         $mol->get_atoms(7), 
         $mol->get_atoms(10), 
         $mol->get_atoms(11), 
         $mol->get_atoms(5),
        ]
        ); 
        

HackaMol::Dihedral=HASH(0x7fed4fe84a38)


In [35]:
my %results = ();
my $gran = 10;

foreach my $t (0 .. $mol->tmax){
    $mol->t($t);
    my $chi3  = ceil($dihe3->dihe_deg/$gran); 
    my $chi1  = ceil($dihe1->dihe_deg/$gran); 
    my $chi1p = ceil($dihe1p->dihe_deg/$gran); 
    $results{$chi3}{$chi1}++;
    $results{$chi3}{$chi1p}++;
}


In [36]:
foreach my $chi3 (sort {$a<=>$b} keys %results){
  foreach my $chi1 (sort {$a<=>$b} keys %{$results{$chi3}}){
   my $count = $results{$chi3}{$chi1};
   printf ("%10.2f %10.2f %10.2f\n",$chi3*$gran,$chi1*$gran,$count*$gran);
  }
}

   -170.00    -130.00      10.00
   -170.00    -110.00      10.00
   -160.00    -120.00      10.00
   -160.00     -80.00      30.00
   -160.00     -70.00      10.00
   -160.00     -60.00      10.00
   -160.00     -20.00      10.00
   -160.00      90.00      10.00
   -160.00     100.00      10.00
   -160.00     130.00      10.00
   -150.00      70.00      10.00
   -150.00      90.00      10.00
   -150.00     110.00      10.00
   -150.00     140.00      10.00
   -140.00     -60.00      10.00
   -140.00      50.00      10.00
   -140.00      70.00      10.00
   -140.00     100.00      10.00
   -130.00     -90.00      20.00
   -130.00     -30.00      20.00
   -130.00      60.00      10.00
   -130.00      80.00      30.00
   -130.00      90.00      20.00
   -130.00     100.00      30.00
   -130.00     110.00      40.00
   -130.00     120.00      20.00
   -130.00     180.00      10.00
   -120.00    -170.00      20.00
   -120.00    -130.00      10.00
   -120.00    -100.00      20.00
   -120.00

In [2]:
my $t1 = time;

my $type = shift or die "pass type: xtals nmrs";
my $work = HackaMol->new( data    => "$type\_clusters",
                          scratch => "results/$type" );

my @pdbs = $work->data->children(qr/\.pdb/);

$work->scratch->mkpath unless $work->scratch->exists;

my @aas = mce_map{
    my $fpdb = $_;
    my $hack = HackaMol->new( name => "hackitup" );
    my @aas  = map  {$_->aa321}
               grep {$_->name eq 'CA' } $hack->read_file_atoms($fpdb);
}@pdbs;
my %aa ;
$aa{$_}++ foreach @aas;
$aa{C} -= 2*scalar(@pdbs); # do not include the disulfide itself

my @a = qw/
L V I Y F W A P G M 
S T Q N H K R D E C
/;

my $t = 0;

$t += $aa{$_} foreach @a;


printf( "%8.3f\n", $aa{$_}/$t) foreach @a;
#printf( "%5s %8.6f %5i\n", $_, $aa1{$_}/$t,  $aa1{$_}) foreach @a;

Error: pass type: xtals nmrs at reply input line 3.


In [3]:
system ("pwd")

/Users/chem_student/Fadilahs_Space/devel/HackaMol-CaseStudy/Survey_PDBdisulfide


0
